In [51]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [63]:
address = pd.read_csv('./address.csv')
address.columns
items = address.loc[:,'식당명']
items = items.tolist()

In [69]:
driver = webdriver.Chrome("./chromedriver")

count = 0
items = address.loc[:,'식당명']
items = items.tolist()
rating_df = pd.DataFrame()
for item in items:
    # 리뷰가 없을 때의 코드
    
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        content_url = reviewbutton.get_attribute("href")    
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img ') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                print(user_review)
                print(rating)
                print(img_url)
                print(user_id)
                print(timestamp)
                row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating,"Timestamp":timestamp}
                row = pd.DataFrame(row, index=[i])
                rating_df = rating_df.append(row,ignore_index=True)
                time.sleep(1)
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,999):
                time.sleep(2)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img ') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating,"Timestamp":timestamp}
                        row = pd.DataFrame(row, index=[i])
                        rating_df = rating_df.append(row,ignore_index=True)
                        time.sleep(1)
        except:
            print("더 이상 리뷰 존재 X")
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        print("식당 존재 x")

식당 존재


<ipython-input-69-42dc37cb8c9e>:23: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[1]) # 새 탭으로 이동


0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자


<ipython-input-69-42dc37cb8c9e>:70: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(tabs[0])


식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
식당 존재
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
식당 존재
5
[<span>쫀득하고 맛있습니다. 막국수도 정말 맛있어요. 콩나물국이 대단합니다. 더 번창하십쇼! </span>]
[<em class="num_rate">5<span class="screen_out">점</span></em>]
[]
[<a class="link_user" data-ismy="false" data-ori="http://th-p.talk.kakao.co.kr/th/talkp/wkE8WImdR0/S9TmFjc4xzIfr3jhaIDh90/3922jc.jpg" data-platform="kakaomap" data-userid="fomd99" data-username="먹보의길" href="javascript:void(0);">먹보의길</a>]
[<span class="time_write">2021.01.16.</span>]
[<span>괜찮아요</span>]
[<em class="num_rate">3<span class="screen_out">점</span></em>]
[]
[<a class="link_user" data-ismy="false" data-ori="http://t1.daumcdn.net/local/kakaomapPhoto/profile/040fc19c68251225bac3df456976b7f18147ce7d?original" data-platform="kakaomap" data-userid="qk8du6" data-username="jinyung" href="javascript:void(0);">jinyung</a>]
[<span class="time_write">2020.10.11.</span>]
[<span>완전 맛집.. ㅁ1친nom임.. 쫀쫀한 족발... 공기밥에 시래기?우거지?국 말아먹으면 대환장... 근데 솔직히 막국수는 노맛..ㅋ 둘째넷째 화요일 쉬니까 꼭 참고하

더 이상 리뷰 존재 X
기본 페이지로 돌아가자
식당 존재
4
[<span>Good :)</span>]
[<em class="num_rate">5<span class="screen_out">점</span></em>]
[]
[<a class="link_user" data-ismy="false" data-ori="http://th-p.talk.kakao.co.kr/th/talkp/wk1TPNM1nz/T3b6V794bW0kdWaN4CJ2zk/neqqad.jpg" data-platform="kakaomap" data-userid="bg9k3u" data-username="정다은" href="javascript:void(0);">정다은</a>]
[<span class="time_write">2019.01.30.</span>]
[<span>남자화장실 칸이 한칸이라 너무 곤욕이었습니다 </span>]
[<em class="num_rate">2<span class="screen_out">점</span></em>]
[]
[]
[<span class="time_write">2017.10.05.</span>]


IndexError: list index out of range

In [45]:
import string 
f = pd.read_csv('./전라북도 전주시_음식점 정보_20201229.csv',encoding = 'utf-8')
# f.head()
f_pung = f.loc[f['지번주소'].str.contains('풍남동')]
# print(f_pung)
f_kyo = f.loc[f['지번주소'].str.contains('교동')]
f_j = [f_pung,f_kyo]
f_hanok = pd.concat([f_pung,f_kyo])
f_hanok.drop(f_hanok.iloc[:,5:], axis=1)
f_hanok.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 1648 to 2368
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   식당명          95 non-null     object 
 1   도로명주소        95 non-null     object 
 2   지번주소         95 non-null     object 
 3   식당위도         95 non-null     float64
 4   식당경도         95 non-null     float64
 5   Unnamed: 5   0 non-null      float64
 6   Unnamed: 6   0 non-null      float64
 7   Unnamed: 7   0 non-null      float64
 8   Unnamed: 8   0 non-null      float64
 9   Unnamed: 9   0 non-null      float64
 10  Unnamed: 10  0 non-null      float64
 11  Unnamed: 11  0 non-null      float64
 12  Unnamed: 12  0 non-null      float64
 13  Unnamed: 13  0 non-null      float64
 14  Unnamed: 14  0 non-null      float64
 15  Unnamed: 15  0 non-null      float64
 16  Unnamed: 16  0 non-null      float64
 17  Unnamed: 17  0 non-null      float64
 18  Unnamed: 18  0 non-null      float64
 19  Unnam

In [49]:
# f_duc = f_hanok.loc[f_hanok['도로명주소'].str.contains('덕진구')]
f_hanok=f_hanok[~f_hanok.도로명주소.str.contains('덕진구')]
f_hanok.info()
f_hanok.to_csv('./address.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 1666 to 2368
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   식당명          89 non-null     object 
 1   도로명주소        89 non-null     object 
 2   지번주소         89 non-null     object 
 3   식당위도         89 non-null     float64
 4   식당경도         89 non-null     float64
 5   Unnamed: 5   0 non-null      float64
 6   Unnamed: 6   0 non-null      float64
 7   Unnamed: 7   0 non-null      float64
 8   Unnamed: 8   0 non-null      float64
 9   Unnamed: 9   0 non-null      float64
 10  Unnamed: 10  0 non-null      float64
 11  Unnamed: 11  0 non-null      float64
 12  Unnamed: 12  0 non-null      float64
 13  Unnamed: 13  0 non-null      float64
 14  Unnamed: 14  0 non-null      float64
 15  Unnamed: 15  0 non-null      float64
 16  Unnamed: 16  0 non-null      float64
 17  Unnamed: 17  0 non-null      float64
 18  Unnamed: 18  0 non-null      float64
 19  Unnam

In [50]:
import os
path = os.getcwd()
os.listdir(path)

['.ipynb_checkpoints',
 'address.csv',
 'chromedriver.exe',
 'graduate_analy.ipynb',
 '전라북도 전주시_음식점 정보_20201229.csv',
 '종설 크롤링.ipynb']